In [19]:
#!pip install ultralytics OpenEXR Imath
from ultralytics import YOLO
import cv2
import OpenEXR
import Imath
import numpy as np

def read_exr_file(file_path):
    # Open the EXR file
    exr_file = OpenEXR.InputFile(file_path)

    # Get the header to extract basic information like resolution
    header = exr_file.header()
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1

    # Print the channel names
    channels = header['channels'].keys()
    print(f"Channels: {channels}")

    # Initialize a dictionary to hold the channel data
    channel_data = {}

    # Read all the channels present in the EXR file
    for channel in channels:
        channel_str = exr_file.channel(channel, Imath.PixelType(Imath.PixelType.FLOAT))
        channel_data[channel] = np.frombuffer(channel_str, dtype=np.float32).reshape((height, width))

    # If the file contains RGB channels, merge them into a single image
    if 'R' in channels and 'G' in channels and 'B' in channels:
        img = np.stack([channel_data['B'], channel_data['G'], channel_data['R']], axis=-1)
    else:
        raise ValueError("RGB channels not found in the EXR file.")

    return img

model = YOLO("/content/best-150.pt")



#image in predict2, text in predict2/labels

#results = model("/content/First.png")
model.predict("/content/00104Left.png", save=True,save_txt=True, conf=0.3)
img=cv2.imread("/content/runs/detect/predict18/00104Left.png")
f=open("/content/runs/detect/predict18/labels/00104Left.txt")
s=len(img)

dep = read_exr_file("/content/00104Left.exr")

dep=cv2.cvtColor(dep,cv2.COLOR_BGR2GRAY)
'''for i in range(s):             #trace between inf and non-inf
  for j in range(s):
    if dep[j,i]<10000000:
      cv2.circle(img,(i,j),2,(255,0,0),thickness=-1)
      break'''

count=1
while True:
  #print("hhhh")
  line =f.readline()
  if not line:
    break
  l=line.split(" ")
  #print(l)
  (x,y)=(int((float(l[1]))*s),int((float(l[2]))*s))
  #cv2.circle(img,(x,y),3,(255,0,0),thickness=-1)

  if (dep[y,x]>=10000000000):
    cv2.putText(img,"inf",(x-10,y+10),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.6,color=(255,255,255),thickness=1)
  else:
    cv2.putText(img,str(round(dep[y,x],2)),(x-10,y+10),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.6,color=(255,255,255),thickness=1)
  print(count, x, y, dep[y,x])
  count+=1

cv2.imwrite("zamn.png",img)

#print(results)


image 1/1 /content/00104Left.png: 640x640 12 rocks, 298.4ms
Speed: 7.5ms preprocess, 298.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict18
1 label saved to runs/detect/predict18/labels
Channels: dict_keys(['B', 'G', 'R'])
1 426 460 1.5296326
2 159 485 1.1076856
3 221 344 3.2535963
4 65 371 2.6483903
5 154 360 2.848288
6 268 220 5.5714436
7 393 250 5.146155
8 396 341 3.3529038
9 13 301 4.081373
10 246 279 4.4526434
11 288 339 3.3308377
12 456 253 4.787533


True